In [ ]:
from matplotlib import pyplot as plt
import matplotlib as mpl
plt.style.use("ggplot")
mpl.rcParams['font.size'] = 16
mpl.rcParams['legend.fontsize'] = 14
from scipy.integrate._ivp.ivp import OdeResult
from scipy.optimize import curve_fit
import seaborn as sns
import numpy as np
import tqdm, os, glob
import itertools

from ars_247_2025_final_project.tram_model import TRAM_Model
from ars_247_2025_final_project.tools import gaussian_pulse, double_sigmoid_pulse

In [ ]:
OUTPUT_DIR = os.path.abspath("..")
OUTPUT_PNG_DIR = os.path.join(OUTPUT_DIR, "Final_Project_PNG")
print(f"Saving files to {OUTPUT_DIR}...")
OUTPUT_NPZ_DIR = os.path.join(OUTPUT_DIR, "npz")
USE_EXISTING_NPZ = False
if len(glob.glob(os.path.join(OUTPUT_NPZ_DIR, "*.npz"))) > 0:
    answer = ""
    while answer != 'y' and answer != 'n':
        answer = input(f"Detected '.npz' file in {OUTPUT_NPZ_DIR}. Would you like to use it as your data? (y/n)")
    if answer == 'y': USE_EXISTING_NPZ = True

In [ ]:
HOUR = 3600
MINUTE = 60
# transcriptional pulse
transcriptional_pulse = {
    'peak_time': 1.5 * HOUR,
    'peak_value': 1,
    'width': 0.5 * HOUR
}

# phosphorylation pulse
phosphorylation_pulse = {
    'peak_time': 1.5 * HOUR // 4,
    'peak_value': 1,
    'width': 0.5 * HOUR // 4 # 45 mins peak phosphorylation pulse
}

# optogenetic pulse
optogenetic_pulse = {
    'peak_time': 1.5 * HOUR // 10, # peak at 9 mins
    'peak_value': 1,
    'width': 0.5 * HOUR // 10 # 3 mins peak activity of pulse
}

system_params = {
    'PROM': 1,
    'V_A': 0.47, #um/s - mean for dynein / kinesin
    'R_M': 2.5, #um - mean for mammalian cells is 2.5; best results w/ 20
    'K_TEV': 0.015, #/s - physiological is ~0.15
    'K_TVMV': 0.015, #/s - physiological is ~0.15
    'K_TRAM_DEG': 0.00001, #/s - got interesting results with this being 0.01; physiological is ~0.00001
    'K_TF_DEG': 0.01 #/s - got interesting results with this being 0.1; physiological is ~0.01 w/ fast degron
}

## Figure 2

In [ ]:
# Creating the linear parameter space for each parameter of interest
linear_parameter_space = np.concat([1 / np.arange(2, 8, 2)[::-1], np.ndarray([1]), np.arange(2, 8, 2)])
speed_ranges = linear_parameter_space * system_params['V_A']
protease_ranges = linear_parameter_space * system_params['K_TEV']
deg_tram_ranges = linear_parameter_space * system_params['K_TRAM_DEG']
deg_tf_ranges = linear_parameter_space * system_params['K_TF_DEG']

if not USE_EXISTING_NPZ:
    # Combinatorially constructing all possible parameter combinations based on linear parameter space
    all_combinations = np.array(list(itertools.product(*[speed_ranges, protease_ranges, deg_tram_ranges, deg_tf_ranges])))
    models_comb = []
    T = 24 * HOUR
    N_DOMAINS = 6
    solns_comb = np.zeros(shape=(len(all_combinations), 4 * N_DOMAINS + 1, T + 5))
    for idx, (speedx, protx, degrx, degtx) in enumerate(tqdm.tqdm(all_combinations)):
        system_params['V_A'] = speedx
        system_params['K_TEV'], system_params['K_TVMV'] = protx, protx
        system_params['K_TRAM_DEG'], system_params['K_TF_DEG'] = degrx, degtx

        this_model = TRAM_Model(
            n_domains=N_DOMAINS,
            pulse='gaussian',
            pulse_params=optogenetic_pulse,
            sys_config=system_params
        )

        models_comb.append(this_model)
        solns_comb[idx,...] = this_model.solve_tram_ivp(0, T)['y']

else:
    print("User elected to use existing NPZ as input data, skipping IVP solving step...")

Stacking results into a solution object and saving to NPZ file

In [ ]:
if not USE_EXISTING_NPZ: 
    npz_save_dict = {
        'speed_ranges': speed_ranges,
        'protease_ranges': protease_ranges,
        'deg_tram_ranges': deg_tram_ranges,
        'deg_tf_ranges': deg_tf_ranges,
        'combinations': all_combinations,
        'solutions': solns_comb
    }
    
    np.savez(
        file=os.path.join(OUTPUT_NPZ_DIR, "linear_parameter_space_results.npz"),
        **npz_save_dict
    )

### Figure 2a